# A Simple Slideshow

Let's use iDOM to create a simple slideshow which changes whenever a user clicks an image.

In [ ]:
import idom

## Some Little Set Up

Since you may be running this notebook on [mybinder.org](https://mybinder.org) we have to do a little bit of work to make sure that your frontend view connects to the right URL. You shouldn't have to change anything here. Be sure to [post an issue](https://github.com/rmorshea/idom/issues) if you experience any problems. 

In [ ]:
from example_utils import localhost, pretty_dict_string

webpage_url = localhost('http', 8765) + '/idom/client/index.html'
websocket_url = localhost('ws', 8765) + '/idom/stream'

print('Get a webpage that streams layout updates via a websocket:')
print(webpage_url)
print()
print('Websockets use this route to stream layout updates:')
print(websocket_url)

# The Code

In [ ]:
@idom.element
async def Slideshow(self, index=0):
    events = idom.Events()

    @events.on("click")
    async def change():
        self.update(index + 1)

    url = f"https://picsum.photos/800/300?image={index}"
    return idom.node("img", src=url, eventHandlers=events)


print(f"TRY IT OUT HERE TOO: {webpage_url}\n")

idom.SimpleServer(Slideshow).daemon("0.0.0.0", 8765, access_log=False)

# Try it Out!

Just click the image to move the slideshow forward.

In [ ]:
idom.display("jupyter", websocket_url)

# What Just Happened?

It looks like magic, but at the core of iDOM are simple dictionary structures that model an HTML layout.

To help you understand what's happening let's see what this dictionary model looks like:

In [ ]:
slideshow = Slideshow()
model = await slideshow.render()
print(pretty_dict_string(model))

This will render the following HTML

```html
<img src='https://picsum.photos/800/300?image={index}' onClick={slideshow-change-callback}/>
```

Where each time the user clicks the rendered image:

1. The `slideshow-change-callback` is triggered and calls `change()`

2. The `change()` function then tells the `Slideshow` element to update with a new `{index}`.

3. A new image is rendered for the user with the updated `src` index.